In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('Production Report June-August 2022-2.xlsx')

In [3]:
df['M/C Brand'].value_counts()

M/C Brand
Fukuhara      3641
JiunnLong     2330
Hangxing      2227
Pailung       2041
Lisky          620
Jiunn Long     581
Santoni        179
Name: count, dtype: int64

In [4]:
df['Body Part'].value_counts().head()

Body Part
Main Fabric Top       4768
Body                  2158
Main Fabric Bottom     932
Body Neck Binding      831
Body part-1            585
Name: count, dtype: int64

In [5]:
df['Booking Type'].value_counts()

Booking Type
Main                    8782
Sample Without Order    2091
Short                    305
Partial                    3
Name: count, dtype: int64

In [6]:
df['Construction'].value_counts()[:20]

Construction
S/J                                 5412
LS/J                                 975
L S/J                                842
Pique                                562
Design Mesh                          539
French Terry Inside Soft Brushed     337
Scuba Interlock                      278
1x1 Rib                              265
1X1 Rib                              236
1X1 Rib [Plated]                     172
Terry                                169
Interlock                            158
Fleece Inside Brush                  150
FLEECE                               136
2X2 RIB                              134
Flat Knit Rib                        131
Fleece                               117
1x1 L -Rib                           102
FRENCH TERRY                          94
Jersey                                88
Name: count, dtype: int64

In [7]:
df['Composition'].value_counts()[:20]

Composition
BCI Cotton 100%                                                     2232
Cotton 100%                                                         2094
100% SD Polyester %                                                  633
BCI Cotton 95% Elastane 5%                                           548
95% Cotton-5% Elastan 100%                                           535
60% CMIA Cotton 40% Polyester %                                      361
95% Cotton-5% Elastan %                                              312
Polyester 100%                                                       295
Organic Cotton 100%                                                  285
76% Cotton 20% Polyester 4% Spandex %                                278
60% Cotton 40% Polyester 100%                                        247
95% Viscose-5% Elastan 100%                                          242
90% Cotton - 10% Viscose %                                           191
100% Cotton %                          

In [8]:
df['Color'].value_counts()

Color
WHITE                                                  497
BLACK                                                  487
PUMA WHITE-01                                          433
PUMA WHITE-COLORO-01                                   361
GREY MARL-03                                           333
                                                      ... 
POINSETTIA                                               1
BLUE SHADOW 17-4020 TCX, MOOD INDIGO - 19-4025 TCX       1
BURNT SIENNA - 17-1544 TCX                               1
CARBON 48531                                             1
INDIGO MARL BROS B0824                                   1
Name: count, Length: 1636, dtype: int64

In [9]:
df['Color Range'].value_counts()

Color Range
White Color                  2641
Melange                      2146
Average Color                1691
Dark Color                   1552
Black Color                  1472
Light Color                  1038
Wash                         1010
Medium Color                  964
,Navy                           7
Melange,Medium Color            2
White Color,Medium Color        1
Name: count, dtype: int64

In [10]:
df['Stich'].value_counts()

Stich
2.85               1161
2.90                907
3.15/1.15           557
12.00/13.00         537
2.88                451
                   ... 
1.70/0.75             1
3.10/1.65/1.10        1
1.55/1.30/0.60        1
3.10/1.100            1
1.70/0.80             1
Name: count, Length: 345, dtype: int64

In [11]:
df.isna().sum()

M/C Brand                   950
Production Date               0
M/C Dia & Gauge               0
Body Part                    52
Booking Type               1388
Yarn Count                   33
Lot                          36
Construction                  0
Composition                  52
Color                         4
Color Range                  45
Stich                         0
Dia                           0
GSM                           0
SHIFT-A                       0
SHIFT-B                       0
SHIFT-C                       0
Without Shift(Outbound)      52
Shift Total                   0
Machine Total               359
Reject Qty                 4448
dtype: int64

In [12]:
#in maximum cases the factory used Fukuhara machine so we can fill the missing values with Fukuhara
df['M/C Brand']= df['M/C Brand'].fillna('Fukuhara')

In [13]:
#it seems the excel file auto convert the datatype to date time type; so lets convert it as string
df['Yarn Count']= df['Yarn Count'].apply(lambda x: str(x.day)+'/'+'1' if type(x).__name__ == 'datetime' else x)

In [14]:
df['Yarn Count'].isna().sum()

33

In [15]:
# the yarn count has 33 missing values- filling the yarn count

### Filling the yarn missing counts: 33 yarn count values are missing, we know yarn count is closely related to GSM and Body Parts, so we will take the missing values from GSM and body parts

In [16]:
#making a dataframe of the missing yarn_count values 
body_part_ =[]
count_ =[]

gsm_ =[]
for body_part, gsm in zip(df[df['Yarn Count'].isna()]['Body Part'],
                         df[df['Yarn Count'].isna()]['GSM']):
    body_part_.append(body_part)
    gsm_.append(gsm)
    try:
        count_.append(df[(df['Body Part'] == body_part) & (df['GSM'] == gsm)]['Yarn Count'].value_counts().index[0])
    except:
        count_.append('30/1')

In [17]:
filling_vlaues_count= pd.DataFrame({'Body_Part': body_part_, 'GSM': gsm_, 'Yarn_count': count_ })
filling_vlaues_count['GSM'] = filling_vlaues_count['GSM'].astype(int)
filling_vlaues_count.head()

,Body_Part,GSM,Yarn_count
0,Main Fabric Top,320,"34/1,20/D,50D/36F"
1,Collar,650,150D/144F
2,Collar,650,150D/144F
3,Collar,650,150D/144F
4,Body,200,"30/1,40/D"


In [18]:
#to extract the count from gsm and body_part
def extract_count(gsm):
    return filling_vlaues_count.apply(lambda x: True if x['GSM'] == gsm else False , axis=1)

In [19]:
filling_vlaues_count[extract_count(160)]['Yarn_count'].values

array(['34/1,20/D', '34/1,20/D', '34/1,20/D', '34/1,20/D', '28/1',
       '34/1,20/D', '28/1', '40/1,20/D'], dtype=object)

In [20]:
filling_vlaues_count[extract_count(320)]['Yarn_count'].values

array(['34/1,20/D,50D/36F', '34/1,20/D,50D/36F'], dtype=object)

In [21]:
df['Yarn_Count']= df.apply(lambda x: filling_vlaues_count[extract_count(int(x['GSM']))]['Yarn_count'].values[0]
         if pd.isna(x['Yarn Count']) else x['Yarn Count'] , axis=1)

In [22]:
df['Yarn_Count'].isna().sum()

0

In [23]:
#lets drop the Yarn Count column
df = df.drop('Yarn Count', axis=1)

### GSM tweaking 

In [24]:
gsm_counts = df['GSM'].value_counts()
# gsm_counts
#it seems some GSM are very rare but close to another available GSM, so lets merge them.
# 156==> 155; 164==>165; 195 ==> 190; 205 == > 200; 283 ==> 285;  295 ==> ; 305 ==> ; 310 ==>305 ;
#410 == > 400;

In [25]:
df['GSM']= df['GSM'].replace([156,164,283 ], [156, 165, 285])

## Filling Body Pats

In [26]:
df.isna().sum()

M/C Brand                     0
Production Date               0
M/C Dia & Gauge               0
Body Part                    52
Booking Type               1388
Lot                          36
Construction                  0
Composition                  52
Color                         4
Color Range                  45
Stich                         0
Dia                           0
GSM                           0
SHIFT-A                       0
SHIFT-B                       0
SHIFT-C                       0
Without Shift(Outbound)      52
Shift Total                   0
Machine Total               359
Reject Qty                 4448
Yarn_Count                    0
dtype: int64

In [27]:
def fill_body_part(x):
    if x['GSM'] == 350:
        return 'Rib'
    elif x['GSM'] == 295:
        return 'Armhole Piping'
    elif x['GSM'] == 170:
        return 'body'
    elif x['GSM'] == 185:
        return 'Pocketing Bottom'
    else:
        return 'Body'


In [28]:
df['Body_part']= df.apply(lambda x: fill_body_part(x) if pd.isna(x['Body Part']) else x['Body Part'], axis=1)

In [29]:
df['Yarn_count_length']= df['Yarn_Count'].apply(lambda x: len(x.split(',')))

In [30]:
# multiple_construction_df =df[df['Yarn_count_length']>1][['Construction','Composition', 'Stich','Yarn_Count','Yarn_count_length'] ]

In [31]:
# multiple_construction_df.groupby('Construction').first().sort_values(by='Yarn_count_length', ascending= False)

In [32]:
# df.isna().sum()

### Filling the Composition

In [33]:
#the composition colum was cleaned on the bassis of missing construction
df['Construction']= df.apply(lambda x: x['Construction'].split(',')[0] if pd.isna(x['Composition'])
                             else x['Construction'], axis=1)

In [34]:
#here we haved taken the composition from the construction 
df['Composition']= df.apply(lambda x: x['Construction'].split(',')[-1]
         if pd.isna(x['Composition']) else x['Composition'], axis=1)

### Machine Total is a cumulative production of a single machine for upto few days. so we can not fill the  missing values of machine total. However,we can infer the mean production by using Pivot_table  

### Machine total is the total production per machine 

In [35]:
df['M/C Brand']=df['M/C Brand'].replace('JiunnLong', 'Jiunn Long')

In [36]:
# pd.pivot_table(df, index=['M/C Brand'], values=['Machine Total'] )

### Lot number is significant in determininign the color and color range, so we will use lot number to fill color and color range 

In [37]:
df['Lot']= df['Lot'].apply(lambda x: str(x).strip())
# df['Color']= df['Color'].apply(lambda x: str(x).strip() if x is None)

In [38]:
lot_number =[]
for i in df[df['Color'].isna()]['Lot']:
    lot_number.append(str(i).strip())

In [39]:
lot_number

['PU-JA3181E', 'PU-801-1,PU-Y223009WR', 'PU-Y223009WR', 'PU-Y223009WR']

In [40]:
df[df['Lot']== 'PU-JA3181E']['Color']

2437                         NaN
4988    LIGHT GRAY HEATHER BC04 
9165         LIGHT GRAY HEATHER 
9441         LIGHT GRAY HEATHER 
Name: Color, dtype: object

In [41]:
df[df['Lot']=='PU-801-1,PU-Y223009WR']['Color Range']

5831    Melange 
Name: Color Range, dtype: object

In [42]:
df[df['Lot']=='PU-Y223009WR']['Color Range']

5837    Melange 
5993    Melange 
Name: Color Range, dtype: object

In [43]:
df[df['Lot']=='PU-Y223009WR']['Color Range']

5837    Melange 
5993    Melange 
Name: Color Range, dtype: object

In [44]:
color_fill_dict = {'PU-JA3181E': 'LIGHT GRAY HEATHER', 'PU-Y223009WR': 'Melange', 
                   'PU-801-1,PU-Y223009WR': 'Melange',
 'PU-Y223009WR': 'Melange'  }

In [45]:
df['Color']= df['Color'].fillna(df['Lot'].apply(lambda x: color_fill_dict[x] if x
                                                       in color_fill_dict.keys() else 0))


### Color Range Fill up

#### filling color range with the value of color as color and color range are very similar

In [46]:
for ind, value in zip(df[df['Color Range'].isna()]['Color'].index, df[df['Color Range'].isna()]['Color'].values):
    df.loc[ind, 'Color Range']= value.strip()
    

In [47]:
df.isna().sum()

M/C Brand                     0
Production Date               0
M/C Dia & Gauge               0
Body Part                    52
Booking Type               1388
Lot                           0
Construction                  0
Composition                   0
Color                         0
Color Range                   0
Stich                         0
Dia                           0
GSM                           0
SHIFT-A                       0
SHIFT-B                       0
SHIFT-C                       0
Without Shift(Outbound)      52
Shift Total                   0
Machine Total               359
Reject Qty                 4448
Yarn_Count                    0
Body_part                     0
Yarn_count_length             0
dtype: int64

### filling body part

In [48]:
df[df['Construction']=='Interlock']['Body Part'].value_counts()

Body Part
Body                  80
Main Fabric Top       76
Main Fabric Bottom     2
Name: count, dtype: int64

In [49]:
df[df['Construction']=='Fleece']['Body Part'].value_counts()

Body Part
Main Fabric Top       54
Body                  26
Main Fabric Bottom    20
Body Hood Pocket      11
Bottom                 3
Top Bottom             2
Body [Top Part]        1
Name: count, dtype: int64

### filling the missing body parts

In [50]:
for ind in (df['Body Part'].isna()).index:
    if df.loc[ind, 'Construction'] == 'Interlock':
        df.loc[ind, 'Body Part'] == 'Body'
    elif df.loc[ind, 'Construction'] == 'Fleece':
         df.loc[ind, 'Body Part'] == 'Main Fabric Top'


In [51]:
df['Booking Type']= df['Booking Type'].fillna('Other')

In [52]:
df.isna().sum()

M/C Brand                     0
Production Date               0
M/C Dia & Gauge               0
Body Part                    52
Booking Type                  0
Lot                           0
Construction                  0
Composition                   0
Color                         0
Color Range                   0
Stich                         0
Dia                           0
GSM                           0
SHIFT-A                       0
SHIFT-B                       0
SHIFT-C                       0
Without Shift(Outbound)      52
Shift Total                   0
Machine Total               359
Reject Qty                 4448
Yarn_Count                    0
Body_part                     0
Yarn_count_length             0
dtype: int64

### data types manipulation

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12569 entries, 0 to 12568
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   M/C Brand                12569 non-null  object 
 1   Production Date          12569 non-null  object 
 2   M/C Dia & Gauge          12569 non-null  object 
 3   Body Part                12517 non-null  object 
 4   Booking Type             12569 non-null  object 
 5   Lot                      12569 non-null  object 
 6   Construction             12569 non-null  object 
 7   Composition              12569 non-null  object 
 8   Color                    12569 non-null  object 
 9   Color Range              12569 non-null  object 
 10  Stich                    12569 non-null  object 
 11  Dia                      12569 non-null  object 
 12  GSM                      12569 non-null  object 
 13  SHIFT-A                  12569 non-null  float64
 14  SHIFT-B               

In [54]:
# df['Production Date']

In [55]:
def dia_strip(x):
    try:
        return int(x)
    except:
        try: 
            return int(x.split('/')[0])
        except:
            try:
                return int(x.split(',')[0])
            except:
                if 'any' in x.lower():
                    return 0
                else:
                    return x

In [56]:
df['m/c_dia']= df['M/C Dia & Gauge'].str.split('X', expand= True)[0].astype(int)
df['m/c_gauge']= df['M/C Dia & Gauge'].str.split('X', expand= True)[1].astype(int)

In [57]:
df['Dia']= df['Dia'].str.strip().replace('ANY', 'any')

In [58]:
def dia_splitter(x):
    try:
        return int(x.split('/')[0])
    except:
        try:
            return int(x.split(',')[0])
        except :
            return x
    

In [59]:
def dia_converter(x):
    try:
        return int(x)
    except:
        return x

In [60]:
df['fabric_dia_1']=df['Dia'].apply(lambda x: dia_splitter(x))

In [61]:
df['fabric_dia_1']= df['fabric_dia_1'].apply(lambda x: dia_converter(x))

In [62]:
def dia_splitter_2(x):
    try:
        return int(x.split('/')[1])
    except:
        try:
            return int(x.split(',')[1])
        except :
            return x
    

In [63]:
df['fabric_dia_2']= df['Dia'].apply(lambda x: dia_splitter_2(x))

In [64]:
df['fabric_dia_2']= df['fabric_dia_2'].apply(lambda x: dia_converter(x))

In [65]:
def contraction(x):
    try:
        return x['fabric_dia_1']/x['m/c_dia']
    except:
        return 0

In [66]:
def contraction_2(x):
    try:
        return x['fabric_dia_2']/x['m/c_dia']
    except:
        return 0

In [67]:
df['contraction_1']= df.apply(lambda x: contraction(x), axis=1)

In [68]:
df['contraction_2']= df.apply(lambda x: contraction_2(x), axis=1)

In [69]:
df['contraction']= (df['contraction_1'] + df['contraction_2'])/2

In [70]:
df['Construction']= df['Construction'].replace('L S/J', 'LS/J')


# Feature Engineering

In [71]:
#number of feeders
def feeder_number(x):
    if (x['M/C Brand'] == 'Fukuhara') & ((x['Construction'] == 'S/J') | (x['Construction'] == 'LS/J')) :
        return x['m/c_dia']*3 + 6
    elif (x['M/C Brand'] == 'Fukuhara') & (x['Construction'] != 'S/J') & (x['Construction'] != 'LS/J')  :
        return x['m/c_dia']*3
    else:
        return x['m/c_dia']*2

In [72]:
df['number_of_feeders']= df.apply(lambda x: feeder_number(x), axis=1 )

In [73]:
df['yarn_count_1'] = df['Yarn_Count'].str.split(',', expand = True)[0].str.split('/', expand = True)[0]

In [74]:
df['yarn_count_1']= df['Yarn_Count'].str.split(',', expand = True)[0]

In [75]:
def count_converter(x):
    try:
        if 'D' not in x:
            return 590.6/int(x[:2])
        elif 'D' in x:
            denier = int(x[:2])
            return denier/9
        else:
            return 0
    except:
        return x

In [76]:
df['yarn_count_tex_1']= df['yarn_count_1'].apply(lambda x: count_converter(x))

In [77]:
# data['yarn_count_1'] = 
df['yarn_count_tex_2']= df['Yarn_Count'].str.split(',', expand = True)[1].apply(lambda x: count_converter(x))


In [78]:
df['yarn_count_tex_3']= df['Yarn_Count'].str.split(',', expand = True)[2].apply(lambda x: count_converter(x))


In [79]:
def stitch_splitter(x):
    try:
        if ',' in x:
            return float(x.split(',')[0])
        else:
            return float(x)
    except:
        return x
    

In [80]:
df['stitch_1']= df['Stich'].str.split('/', expand = True)[0].str.strip().apply(lambda x: stitch_splitter(x))


In [81]:
# data['stitch_1']= 
df['stitch_2']= df['Stich'].str.split('/', expand = True)[1].str.strip().apply(lambda x: stitch_splitter(x))


In [82]:
df['stitch_3']= df['Stich'].str.split('/', expand = True)[2].str.strip().apply(lambda x: stitch_splitter(x))


In [83]:
def calculating_tightness_factor(x):
    stitch = x['stitch_1']
    tf = np.sqrt(x['yarn_count_tex_1'])/ stitch
    return tf

In [84]:
df['tightness_factor']= df.apply(lambda x: calculating_tightness_factor(x), axis=1)

# Sorting 

In [85]:
def composition_shorter(x):
    if 'cotton' in x.lower() and '100' in x.lower():
        return 'Cotton'
    elif 'polyes' and '100' in x.lower():
        return'Polyester'
    elif 'cvc' in x.lower():
        return 'Polyester'
    elif ('cotton' or 'ctn') in x.lower() and ('polyester' or 'poly' ) in x.lower():
        return 'Cotton_Polyester'
    elif 'ctn' and 'poly' in x.lower():
        return 'Cotton_Polyester'
    elif 'cotton' and 'span' in x.lower():
        return 'Cotton_Elastan'
    elif ('cotton' or 'ctn' in x.lower()) and ('elast' or 'ly' or 'span') in x.lower() :
        return 'Cotton_Elastan'
    
    elif 'cotton' and 'viscose' in x.lower():
        return 'Cotton_Viscose'
    else:
        return 'other_cotton_blend'

In [86]:
df['composition']= df['Composition'].apply(lambda x: composition_shorter(x))

In [87]:
df= df.drop('Composition', axis=1)

In [88]:
def construction_shorter(x):
    if 'ri' in x.lower():
        return 'Rib'
    elif 'fleece' in x.lower() or 'terry' in x.lower():
        return 'Fleece_terry'
    elif ('s/j' in x.lower())  and ('l' not in x.lower()) :
        return 'S/J'
    elif 'jers' in x.lower():
        return 'S/J'
    elif 'l' and 's/j' in x.lower():
        return 'LS/J'
    elif 'inter' in x.lower():
        return 'Interlock'
    elif 'mesh' in x.lower():
        return 'Mesh'
    elif 'pique' in x.lower():
        return 'Pique'
    else:
        return 'other'

In [89]:
# data['composition_sorted']= 
df['construction']= df['Construction'].apply(lambda x: construction_shorter(x))

In [90]:
df= df.drop('Construction', axis=1)

In [91]:
def body_part_shorter(x):
    if 'Main Fabric Bottom' == x or 'bottom' in x.lower():
        return 'Bottom'
    elif 'top' in x.lower():
        return 'Top'
    elif 'body' in x.lower() :
        return 'Body'
    elif 'rib' in x.lower() :
        return 'Rib'
    elif 'neck' in x.lower():
        return 'Rib'
    elif 'collar' in x.lower():
        return 'Rib'
    elif 'cuff' in x.lower():
        return 'Rib'
    else: 
        return 'other'

In [92]:
df['body_part']= df['Body_part'].apply(lambda x: body_part_shorter(x))

In [93]:
df= df.drop(['Body Part', 'Body_part'], axis=1)

In [94]:
def stitch_2_cleaner(x):
    try:
        return float(x)
    except:
        try:
            return float(x.strip('.'))
        except:
            try:
                if 'CM' in x:
                    try:
                        return float(x[:3])*10
                    except:
                        return x.split('-')[1][:3] 
                
                elif '..' in x:
                    return float(x.replace('..','.'))                    
            except :
                return x
                

In [95]:
df['stitch_2'] = df['stitch_2'].apply(lambda x: stitch_2_cleaner(x)).astype(float)

In [96]:
df['GSM']= df['GSM'].astype(int)

In [97]:
df.head()

,M/C Brand,Production Date,M/C Dia & Gauge,Booking Type,Lot,Color,Color Range,Stich,Dia,GSM,...,yarn_count_tex_1,yarn_count_tex_2,yarn_count_tex_3,stitch_1,stitch_2,stitch_3,tightness_factor,composition,construction,body_part
0,Fukuhara,01-06-2022,52 X 14,Main,MGF-SS-8,PURE WHITE 00010,White Color,4.75,52,700,...,24.608333,NaN,NaN,4.75,NaN,NaN,1.044353,Cotton,Rib,Rib
1,Fukuhara,01-06-2022,52 X 14,Main,MGF-SS-8,PURE WHITE 00010,White Color,4.75,52,700,...,24.608333,NaN,NaN,4.75,NaN,NaN,1.044353,Cotton,Rib,Rib
2,Fukuhara,01-06-2022,52 X 14,Main,MGF-63,CHARCOAL HEATHER - 00009,Wash,4.75,52,700,...,24.608333,NaN,NaN,4.75,NaN,NaN,1.044353,Cotton,Rib,Rib
3,Fukuhara,01-06-2022,52 X 14,Main,"BF-22-10200,BF-22-12463",RIBBON RED[19-1663 TCX]; 5MM TIPPING-WHITE,Wash,4.70,52,700,...,24.608333,NaN,NaN,4.70,NaN,NaN,1.055464,Cotton,other,Rib
4,Fukuhara,01-06-2022,52 X 14,Main,BF-22-12463,RIBBON RED[19-1663 TCX]; 5MM TIPPING-WHITE,Wash,4.70,52,700,...,24.608333,NaN,NaN,4.70,NaN,NaN,1.055464,Cotton,other,Rib


In [98]:
df= df.rename(columns={'Stich': 'stitch'})

In [99]:
df.isna().sum()

M/C Brand                      0
Production Date                0
M/C Dia & Gauge                0
Booking Type                   0
Lot                            0
Color                          0
Color Range                    0
stitch                         0
Dia                            0
GSM                            0
SHIFT-A                        0
SHIFT-B                        0
SHIFT-C                        0
Without Shift(Outbound)       52
Shift Total                    0
Machine Total                359
Reject Qty                  4448
Yarn_Count                     0
Yarn_count_length              0
m/c_dia                        0
m/c_gauge                      0
fabric_dia_1                   0
fabric_dia_2                   0
contraction_1                  0
contraction_2                  0
contraction                    0
number_of_feeders              0
yarn_count_1                   0
yarn_count_tex_1               0
yarn_count_tex_2            7697
yarn_count

In [101]:
df.to_csv('knitting_dataset_cleaned.csv', index= False)